In [ ]:
# Give instructions for your agent
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write short, concise cold emails that are likely to get a response."

In [ ]:
# Import libraries to create your agents
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

In [ ]:
# Create your agents
sales_agent1 = Agent(
    name="Serious Sales Agent", 
    instructions=instructions1, 
    model='gpt-4o-mini'
    )

sales_agent2 = Agent(
    name="Witty Sales Agent", 
    instructions=instructions2, 
    model='gpt-4o-mini'
    )

sales_agent3 = Agent(
    name="Busy Sales Agent", 
    instructions=instructions3, 
    model='gpt-4o-mini'
    )
    

In [ ]:
# Test using a single sales agent 1
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)


In [ ]:
import asyncio

In [ ]:
# You can launch multiple agents in parallel
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    for output in outputs:
        print(output + "\n\n")
        

https://platform.openai.com/traces

In [ ]:
# Use tools to enable some conditional logic

# First notice that you can include tool functions in the agent

sales_agent1

In [ ]:
# To include tool in tools=[] of the agent
# Wrap your function with the decorator
# @function_tool
# eliminate putting everything into JSON

from sendgrid import from_email, sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import os

@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = 'krisdas8@gmail.com'
    to_email='krisdas8@gmail.com'
    content = Content(mime_type='text/plain', content=body)
    mail = Mail(from_email=from_email, to_emails=to_email, subject="Sales email", plain_text_content=content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}


In [ ]:
send_email(body="This is a test email")

In [ ]:
send_email

In [ ]:
# Turn agent into tools

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Writes a cold sales email")
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Writes a cold sales email")
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Writes a cold sales email")

In [ ]:
tool1

In [ ]:
tools = [tool1, tool2, tool3, send_email]

tools

In [ ]:
instructions = """
You are a Sales Manager at ComplAI.  Your job is to select the best cold sales email using the sales_agent tool.

Follow these steps carefully:
1. Generate drafts: Use all 3 sales_agent tools to each draft 1 unique cold sale email.  Do not proceed until all sales agent completed the draft.
2. Evaluate and select: Evaluate and select the best draft that you think will have the highest response from the email recipient.
3. Use the send_email tool to send the selected email.

Crucial rules:
- You must use the sales agent tools to generate the draft -- do not write them yourself.
- You must use send ONE email using the send email tool -- never more than one.
"""

sales_manager_agent = Agent(name="sales_manager", instructions=instructions, tools=tools, model='gpt-4.1-nano')

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(starting_agent=sales_manager_agent, input=message)

In [ ]:
# Email subject writer agent
subject_instruction = "You are given a cold sale email text. You will write a subject for that email text that will get the most attention from the email recipient."
subject_writer = Agent(name="subject writer", instructions=subject_instruction, model='gpt-4.1-nano')
subject_tool = subject_writer.as_tool(tool_name='subject writer tool', tool_description='writes a subject for the cold sale email')

# Email text to html conversion agent
html_instruction = "You are given an email text that may be in a Markdown format.  You will convert that text into a html format that is beautifully laid out for easy read."
html_writer = Agent(name='html writer', instructions=html_instruction, model='gpt-4.1-nano')
html_tool = html_writer.as_tool(tool_name='html writer tool', tool_description='converts the email text format into html format')


In [ ]:
from typing import Dict

def send_html_email(subject: str, body: str) -> Dict[str, str]:
    """ Send out a html email with subject to potential client """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email("krisdas8@gmail.com")
    to_email = To("krisdas8@gmail.com")
    content = Content(mime_type='text/html', content=body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
email_manager_instructions = "You are an email formatter and sender.  "
email_manager_instructions += "You receive an email body message in text format.  "
email_manager_instructions += "You will use the subject_tool to get the subject for the email text.  "
email_manager_instructions += "You will use the html_tool to convert the email body text into html format.  "
email_manager_instructions += "Finally you will use send_html_email tool to send the email with the subject and html body.  "

tools = [subject_tool, html_tool, send_html_email]

email_manager_agent = Agent(
    name="Email Manager", 
    instructions=email_manager_instructions,
    tools=tools,
    handoff_description="Convert an email to HTML and send it"
    )

In [ ]:
tools = [tool1, tool2, tool3]
hand_off = [email_manager_agent]

print(tools)
print(hand_off)

In [ ]:
instructions = """
You are a Sales Manager at ComplAI.  Your job is to select the best cold sales email using the sales_agent tool.

Follow these steps carefully:
1. Generate drafts: Use all 3 sales_agent tools to each draft 1 unique cold sale email.  Do not proceed until all sales agent completed the draft.
2. Evaluate and select: Evaluate and select the best draft that you think will have the highest response from the email recipient.
3. Handoff for sending email: Use the 'Email Manager' agent to send out email.

Crucial rules:
- You must use the sales agent tools to generate the draft -- do not write them yourself.
- You must use ONE email using the Email Manager -- never more than one.
"""

sales_manager_agent = Agent(
    name="Sales Manager",
    instructions=instructions, 
    tools=tools, 
    handoffs=hand_off,
    model='gpt-4.1-nano')

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Automated SDR"):
    result = await Runner.run(starting_agent=sales_manager_agent, input=message)